Country table 

```sql
CREATE TABLE IF NOT EXISTS `paulkamau.WorldBank_Health.countries_data` as 
SELECT 
country_code, short_name, currency_unit, income_group, national_accounts_base_year, sna_price_valuation, system_of_national_accounts, system_of_trade, government_accounting_concept
 FROM `bigquery-public-data.world_bank_health_population.country_summary` 
where system_of_trade is not null
LIMIT 1000;

```


Pivot query. Convert the row values into column values 

original table we want: 
  `bigquery-public-data.world_bank_health_population.health_nutrition_population`

New pivot table with everything: 
--countries_data_complete

```sql

DECLARE IndicatorCodes STRING;
SET IndicatorCodes = (
  SELECT 
    CONCAT('("', STRING_AGG(DISTINCT replace(indicator_code,".","_"), '", "'), '")'),
  FROM `bigquery-public-data.world_bank_health_population.health_nutrition_population`
);


EXECUTE IMMEDIATE format("""
SELECT * FROM
(
  SELECT country_name, country_code, replace(indicator_code,".","_") as ic, indicator_name,value 
  FROM `bigquery-public-data.world_bank_health_population.health_nutrition_population`
) 
PIVOT
(
  MAX(value)
  FOR ic in %s
)
ORDER BY country_name ASC
""",
IndicatorCodes);


```



What’s the average age of first marriages for females around the world?

```sql
SELECT
  country_name,
  country_code,
  ROUND(AVG(value),2) AS avg_age_female_marriage
FROM
  `bigquery-public-data.world_bank_health_population.health_nutrition_population`
WHERE
  indicator_code = "SP.DYN.SMAM.FE" --Age at first marriage, female
GROUP BY
  country_name,country_code
ORDER BY
  avg_age_female_marriage;
```


Get the distinct indicators used in the entire datasets 

```sql
SELECT * FROM `paulkamau.WorldBank_Health.Indicators_code_desc` LIMIT 1000
```
*results*


--

```sql
CREATE TABLE IF NOT EXISTS `paulkamau.WorldBank_Health.age_fe_marriage` as 
SELECT
  country_name,
  country_code,
  year,
  value AS age_female_marriage
FROM
  `bigquery-public-data.world_bank_health_population.health_nutrition_population`
WHERE
  indicator_code = "SP.DYN.SMAM.FE" --Age at first marriage, female;
```


sql to replace values 

```sql

UPDATE `paulkamau.WorldBank_Health.Indicators_code_desc` 
SET indicator_code_new = REPLACE(indicator_code_new,'.','_')
where indicator_code = indicator_code_new;

```